# Практическое задание 2
## Keras
#### 1) Попробуйте обучить, нейронную сеть на Keras (рассмотренную на уроке) на датасете MNIST с другими параметрами.<br>Опишите в комментарии к уроку - какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

### Решение

In [1]:
import numpy as np
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from tensorflow.keras import regularizers

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 784), (60000,), (10000, 784), (10000,))

Итоговая модель 

In [2]:
model = Sequential([
  Dense(128, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), activation='softmax'),
])

model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [3]:
history = model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=20,
  batch_size=128,
)

model.evaluate(
  test_images,
  to_categorical(test_labels)
)

Epoch 1/20
469/469 [==============================] - 1s 3ms/step - loss: 0.4116 - accuracy: 0.8809
Epoch 2/20
469/469 [==============================] - 1s 3ms/step - loss: 0.2103 - accuracy: 0.9375
Epoch 3/20
469/469 [==============================] - 1s 3ms/step - loss: 0.1518 - accuracy: 0.9551
Epoch 4/20
469/469 [==============================] - 1s 3ms/step - loss: 0.1212 - accuracy: 0.9644
Epoch 5/20
469/469 [==============================] - 1s 3ms/step - loss: 0.1048 - accuracy: 0.9689
Epoch 6/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0907 - accuracy: 0.9735
Epoch 7/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0822 - accuracy: 0.9759
Epoch 8/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0710 - accuracy: 0.9791
Epoch 9/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0664 - accuracy: 0.9798
Epoch 10/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0608 - accuracy: 0.9817

[0.08421438932418823, 0.9775000214576721]

In [4]:
# Evaluate the model.
loss, accuracy = model.evaluate(test_images, to_categorical(test_labels))
print(f'Точность = {accuracy * 100:.2f}%')

313/313 [==============================] - 0s 925us/step - loss: 0.0842 - accuracy: 0.9775
Точность = 97.75%


### Результат
Добавил регуляризацию на последнем слое, так как обнаружил переобучение.

Также увеличил число нейронов на втором скрытом слое.

И число эпох, так как модель не успевала выйти на плато.

В дальнейшем улучшить модель можно, использую сверточные слои.

#### 2) Поработайте с документацией Keras. Попробуйте найти полезные команды Keras неразобранные на уроке.

Вывод весов и смещений по каждому слою

In [5]:
for i, layer in enumerate(model.layers):
  weights, biases = layer.get_weights()
  print(f'Слой {i + 1}')
  print(f'\tВеса:\n {weights}')
  print(f'\tСмещения:\n {biases}')
  print('-' * 100)

Слой 1
	Веса:
 [[-0.03025347  0.0707653   0.01721656 ...  0.02573899  0.04654495
   0.06349827]
 [-0.04084061  0.03234503  0.02283003 ... -0.06230386  0.09616815
   0.05405983]
 [ 0.01269457 -0.01169405 -0.01743331 ...  0.0200057   0.03145126
  -0.01965624]
 ...
 [-0.06283481 -0.07567824 -0.01782317 ...  0.03055907  0.02923272
   0.09709561]
 [-0.00724233  0.01780426  0.00074238 ... -0.0903021   0.07196596
   0.09709181]
 [-0.02395929  0.07448041 -0.02686897 ... -0.0581077  -0.00203207
   0.02007199]]
	Смещения:
 [ 0.00447622  0.00260497 -0.00391595  0.00808433 -0.01517757  0.00743363
 -0.01043962 -0.01292177  0.00246384 -0.00739156  0.02557157  0.00049266
 -0.00384702 -0.01064445  0.00154562 -0.01666865 -0.01241393 -0.00484069
 -0.01619964 -0.0313551  -0.00560535 -0.01839563 -0.00964593 -0.00506236
 -0.00741317 -0.0011305   0.00533494 -0.00715064 -0.00997735 -0.00599707
  0.00207435 -0.008477   -0.01723938  0.01219914 -0.01003345 -0.00515101
 -0.00328121  0.01304124 -0.01030349  0.012

Метод fit возвращает историю.
А в истории есть атрибут history хранится история в виде словаря

In [6]:
history.history

{'accuracy': [0.8809333443641663,
  0.9375333189964294,
  0.955133318901062,
  0.9644333124160767,
  0.9688833355903625,
  0.9734833240509033,
  0.9759166836738586,
  0.9790833592414856,
  0.9797999858856201,
  0.9816833138465881,
  0.9835500121116638,
  0.9845166802406311,
  0.9868666529655457,
  0.9872000217437744,
  0.9879833459854126,
  0.9880499839782715,
  0.9898999929428101,
  0.9915500283241272,
  0.9909666776657104,
  0.9922333359718323],
 'loss': [0.41159605979919434,
  0.2103179693222046,
  0.15182283520698547,
  0.1212158203125,
  0.10476037859916687,
  0.09072805941104889,
  0.08222336322069168,
  0.07102131098508835,
  0.06644511967897415,
  0.0607583187520504,
  0.056857239454984665,
  0.051164139062166214,
  0.04568978026509285,
  0.043670035898685455,
  0.04105738550424576,
  0.03987651690840721,
  0.03557518124580383,
  0.030783018097281456,
  0.031618908047676086,
  0.028704093769192696]}